## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數

Using TensorFlow backend.


In [2]:
num_class=10 #設定分類數量

#讀取資料進行前處理
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

input_shape=x_train.shape[1:]

x_train=x_train.astype('float32')/255
x_test=x_test.astype('float32')/255


#對label進行one-hot encode
y_train=keras.utils.to_categorical(y_train,num_class)
y_test=keras.utils.to_categorical(y_test,num_class)


#學習率動態調整
def lr_schedule(epoch):
    lr=1e-3
    if epoch>180:
        lr*=0.5e-3
    elif epoch>160:
        lr*=1e-3
    elif epoch>120:
        lr*1e-2
    elif epoch>80:
        lr*1e-1
    print('lr is {}'.format(lr))
    return lr

In [3]:
n=6
depth=6*n+2

#建立resnet model
model = resnet(input_shape, depth, num_class) 
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()

lr is 0.001
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
________________________________________________________________________________

In [4]:
lr_scheduler=LearningRateScheduler(lr_schedule)
lr_reducer=ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks=[lr_scheduler,lr_reducer]
print('Using real-time data augmentation.')

Using real-time data augmentation.


In [5]:
augment_generator = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

In [6]:
batch_size = 512
epochs = 1
history=model.fit_generator(augment_generator.flow(x_train,y_train,batch_size=batch_size),
                           steps_per_epoch=int(len(x_train)/batch_size),
                           epochs=epochs,
                            verbose=1,
                            validation_data=(x_test,y_test),
                            callbacks=callbacks
                           )

Epoch 1/1
lr is 0.001
97/97 [==============================] - 2309s 24s/step - loss: 0.7775 - accuracy: 0.9082 - val_loss: 0.7480 - val_accuracy: 0.8955


In [7]:
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# Collect results
train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]
train_acc = model.history.history["accuracy"]
valid_acc = model.history.history["val_accuracy"]

Test loss: 0.7480220772743225
Test accuracy: 0.8955203294754028
